In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

## Use Beautiful Soup to Get Match & Team Details (Upcoming Only)

In [112]:
url = "https://www.oddsportal.com/rugby-union/world/super-rugby/"
html = urlopen(url, timeout = 5)

In [113]:
soup = BeautifulSoup(html, 'lxml')

In [114]:
soup.find_all('tr',{'xeid':True})

[<tr class="odd deactivate" xeid="nBHtcIn5"><td class="table-time datet t1556946900-1-1-0-0"></td><td class="name table-participant"><a href="/rugby-union/world/super-rugby/hurricanes-rebels-nBHtcIn5/">Hurricanes - Rebels</a></td><td class="center bold table-odds table-score">29:19</td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr class="deactivate" xeid="0ELpdx2B"><td class="table-time datet t1556955300-1-1-0-0"></td><td class="name table-participant"><a href="/rugby-union/world/super-rugby/highlanders-waikato-chiefs-0ELpdx2B/">Highlanders - Chiefs</a></td><td class="center bold table-odds table-score">31:31</td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr class="odd deactivate" xeid="rwBkedHH"><td class="table-time datet t1556963100-

In [115]:
games = soup.find_all(attrs={'class':'name table-participant'})

In [143]:
home_team = []
away_team = []

In [144]:
for game in games:
    teams = str(game).split('>')[2].split('<')[0].split(' - ')
    home_team.append(teams[0])
    away_team.append(teams[1])

## Use REST Service for Odds

In [69]:
import json
import requests
import objectpath

In [21]:
s = requests.session()
request_headers = {
    'Referer': 'https://www.oddsportal.com/rugby-union/world/super-rugby/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}

In [22]:
get_odds = s.get('https://fb.oddsportal.com/ajax-sport-country-tournament/8/z36c0gcb/X0/1',
                  headers = request_headers)

In [45]:
text_odds = get_odds.text[72:-2]

In [47]:
json_odds = json.loads(text_odds)

In [70]:
json_tree = objectpath.Tree(json_odds['d'])

In [200]:
odds = tuple(json_tree.execute('$..avg'))
# odds all appear twice for some reason, so we take just half of them
odds = odds[:np.int(len(odds)/2)]

In [181]:
one_list = []
x_list = []
two_list = []

In [183]:
# take every 3 rows and put them into 1 row
for i in range(len(odds)):
    if i%3==0:
        one_list.append(odds[i])
    if i%3==1:
        x_list.append(odds[i])
    if i%3==2:
        two_list.append(odds[i])

In [192]:
# concatenate info into dataframe
df = pd.DataFrame({'Home Team':home_team,
                    'Away Team':away_team,
                    'Home Odds':one_list[:5],
                    'Draw Odds':x_list[:5],
                    'Away Odds':two_list[:5]})

In [194]:
df.head()

,Home Team,Away Team,Home Odds,Draw Odds,Away Odds
0,Hurricanes,Rebels,1.06,41.69,8.86
1,Highlanders,Chiefs,1.24,25.60,4.22
2,Brumbies,Blues,1.23,26.94,4.27
3,Bulls,Waratahs,1.23,26.97,4.14
4,Jaguares,Stormers,1.80,20.44,2.06
